<h1 style="color:green">Conway Pipeline Runner - Branch lifecycle</h1>
<p style="color:green">This notebook contains commands to manage the conway repos. In particular:</p>
    <li style="color:green"> Cross-project consistency - all are committed</li>
    <li style="color:green"> Local-remote consistency - on same commit</li>
    <li style="color:green"> Production branch isolation - uses the right tag/commit</li>
        
<p>We use <a href="https://gitpython.readthedocs.io/en/stable/tutorial.html">GitPython commands</a></p>

In [7]:
import os                                                               as _os
import sys
sys.path.append(_os.path.dirname(_os.getcwd())) # So we can import tvm_notebook_utils
import chassis_nb_utils
NBU                       = chassis_nb_utils.Chassis_NB_Utils()
DFU                       = NBU.DFU
T0                        = NBU.time.perf_counter()

CONWAY installation:                conway_fork    
Jupyter using repo[branch]:      conway.ops[alejandro-fin]    
Installation path:               C:\Alex\Code\conway_fork    
Application:                     <class 'chassis_nb_application.Chassis_NB_Application'>    


In [2]:
DEV_ROOT        = "C:/Alex/Code/conway_fork"
OPERATE_ROOT    = "C:/Alex/Code/conway_fork-operate"
REMOTE_ROOT     = "https://github.com/alejandro-fin/"

In [3]:
CRB             = NBU.Chassis_RepoBundle()
[r.name for r in CRB.bundled_repos()]

['conway.svc',
 'conway.docs',
 'conway.test',
 'conway.scenarios',
 'conway.ops',
 'conway.acceptance',
 'conway.acceptance_docs']

<h1 style="color:blue">Developer workflow</h1>
<p style="color:blue">This is to do feature work:</p>
<li style="color:blue">Choose feature branch</li>
<li style="color:blue">See status at any time</li>
<li style="color:blue">Commit to branch at any time</li>
<li style="color:blue">Deliver to integration when feature is complete</li>

In [4]:
CONWAY_LOCAL_REPOS        = ["conway.svc", "conway.acceptance", "conway.ops", "conway.test", "conway.scenarios"]
CONWAY_LOCAL_BUNDLE       = NBU.RepoBundleSubset(CRB, CONWAY_LOCAL_REPOS)

dev_admin       = NBU.BranchLifecycleManager(local_root=DEV_ROOT, remote_root=REMOTE_ROOT, repo_bundle=CONWAY_LOCAL_BUNDLE)

<h3 style="color:blue">Choose what to work on</h3>

In [5]:
FEATURE_BRANCH  = "alejandro-fin"

In [6]:
dev_admin.work_on_feature(FEATURE_BRANCH)


[176.54 sec]		
-----------conway.svc-----------

[177.23 sec]		Checkout 'alejandro-fin':
Your branch is up to date with 'origin/alejandro-fin'.

[177.84 sec]		
-----------conway.test-----------

[178.49 sec]		Checkout 'alejandro-fin':
Your branch is up to date with 'origin/alejandro-fin'.

[179.14 sec]		
-----------conway.scenarios-----------

[179.85 sec]		Checkout 'alejandro-fin':
Your branch is up to date with 'origin/alejandro-fin'.

[180.44 sec]		
-----------conway.ops-----------

[181.08 sec]		Checkout 'alejandro-fin':
Your branch is up to date with 'origin/alejandro-fin'.

[181.70 sec]		
-----------conway.acceptance-----------

[182.35 sec]		Checkout 'alejandro-fin':
Your branch is up to date with 'origin/alejandro-fin'.


<h3 style="color:blue">Refresh from integration</h3>
<p style="color:blue">This ensures we will work on top of what others have previously delivered</p>

In [ ]:
dev_admin.refresh_from_integration(FEATURE_BRANCH)

<h3 style="color:blue">Status</h3>

In [ ]:
with NBU.Profiler("Gathering repo information"):
    stats_df    = dev_admin.repo_stats() 
stats_df

<h3 style="color:blue">Commit and push</h3>
<p style="color:blue">Push is to remote feature branch, so it is done for backup purposes</p>

In [ ]:
COMMIT_MSG                         = "[" + FEATURE_BRANCH + "] " + ""

COMMIT_MSG

In [ ]:
with NBU.Profiler("Committing work in '" + FEATURE_BRANCH + "'"):
    dev_admin.commit_feature(FEATURE_BRANCH, COMMIT_MSG)

<h3 style="color:blue">Deliver to integration</h3>

In [ ]:
# This should only be done after all tests are passing in the feature branch
#
with NBU.Profiler("Delivering to integration"):
    dev_admin.complete_feature(FEATURE_BRANCH, remove_feature_branch=False)

<h3 style="color:blue">Move from integration to standby branch</h3>
<li style="color:blue">This is to prevent forgetting that integration is checked out and inadvertently change it</li>

In [ ]:
STANDBY_BRANCH = "ah-dev"
dev_admin.work_on_feature(STANDBY_BRANCH)

In [ ]:
dev_admin.refresh_from_integration(STANDBY_BRANCH)

<h1 style="color:green">Release workflow</h1>
<p style="color:green">Publishes a release:</p>
<li style="color:green">PR from integration to master</li>
<li style="color:green">Update operate branch from master</li>

In [ ]:
release_admin       = NBU.VM_BranchLifecycleManager(local_root = OPERATE_ROOT, remote_root =REMOTE_ROOT)

<h3 style="color:green">PR from integration to master</h3>

In [ ]:
# This should only be done after the production-like simulation passes successfully
if True:
    release_admin.pull_request_integration_to_master()

In [ ]:
if True:
    release_admin.publish_release()

<h1 style="color:purple">Operate workflow</h1>

<h3 style="color:purple">Status</h3>

In [ ]:
operate_admin   = NBU.VM_BranchLifecycleManager(local_root = OPERATE_ROOT, remote_root =REMOTE_ROOT)

In [ ]:
with NBU.Profiler("Gathering operate repo information"):
    stats_df    = operate_admin.repo_stats() 
stats_df

<h3 style="color:purple">Commit and push</h3>

In [ ]:
COMMIT_MSG                         = "Hot fix in operate"
COMMIT_MSG

In [ ]:
OPERATE_BRANCH                     = NBU.VM_RepoAdministration.OPERATE_BRANCH

with NBU.Profiler("Committing work in '" + OPERATE_BRANCH + "'"):
    operate_admin.commit_feature(OPERATE_BRANCH, COMMIT_MSG)

<h3 style="color:purple">Publish a hot fix</h3>
<p style="color:purple">This flow from operate to master and from there to integration</p>

In [ ]:
with NBU.Profiler("Publishing hot fix"):
    operate_admin.publish_hot_fix()

<h1 style="color:red">Removing a branch</h1>

In [ ]:
BRANCH_TO_REMOVE = "story_xyz"

In [ ]:
if False:
    with NBU.Profiler("Removing branch"):
        dev_admin.remove_feature_branch(BRANCH_TO_REMOVE)

<h1 style="color:brown">Scratch</h1>